In [3]:
!pip install numpy
!pip install pandas
!pip install math
!pip install scikit-learn
!pip install tensorflow

ERROR: Could not find a version that satisfies the requirement math (from versions: none)
ERROR: No matching distribution found for math
  Using cached scikit_learn-1.5.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached scipy-1.14.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.5.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.4 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.14.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (41.1 MB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)


In [4]:
import numpy as np
#import pyeeg as pe
import pickle as pickle
import pandas as pd
import math

from sklearn import svm
from sklearn.preprocessing import StandardScaler 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

import os
import tensorflow as tf
import time
from sklearn.metrics import accuracy_score, f1_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report


2024-08-18 00:19:16.818339: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-18 00:19:16.820910: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-18 00:19:16.824863: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-18 00:19:16.836526: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-18 00:19:16.855498: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

## 4.Regression
### 0. Loading Training and Testing dataset

In [5]:
with open('out_psds_data_training_2s.npy', 'rb') as fileTrain:
    X_2s  = np.load(fileTrain)
    
with open('out_psds_label_training_2s.npy', 'rb') as fileTrainL:
    Y_2s  = np.load(fileTrainL)

ss = StandardScaler()
X_2s = ss.fit_transform(X_2s)
Z_2s = np.ravel(Y_2s[:, [1]])

Arousal_Train_2s = np.ravel(Y_2s[:, [0]])
Valence_Train_2s = np.ravel(Y_2s[:, [1]])
Domain_Train_2s = np.ravel(Y_2s[:, [2]])

print(Arousal_Train_2s)


with open('out_psds_data_testing_2s.npy', 'rb') as fileTrain:
    M_2s  = np.load(fileTrain)
    
with open('out_psds_label_testing_2s.npy', 'rb') as fileTrainL:
    N_2s  = np.load(fileTrainL)

M_2s = ss.transform(M_2s)
L_2s = np.ravel(N_2s[:, [1]])

Arousal_Test_2s = np.ravel(N_2s[:, [0]])
Valence_Test_2s = np.ravel(N_2s[:, [1]])
Domain_Test_2s = np.ravel(N_2s[:, [2]])


[1 1 5 ... 2 4 3]


In [6]:
# Binarize Arousal_Test_2s with threshold 3
Arousal_Test_2s_binary = np.where(Arousal_Test_2s >= 3, 1, 0)
Arousal_Train_2s_binary = np.where(Arousal_Train_2s >= 3, 1, 0)
# Binarize Valence_Train_2s with threshold 3
Valence_Train_2s_binary = np.where(Valence_Train_2s >= 3, 1, 0)
# Binarize Valence_Test_2s with threshold 3
Valence_Test_2s_binary = np.where(Valence_Test_2s >= 3, 1, 0)
# Binarize Domain_Test_2s with threshold 3
Domain_Test_2s_binary = np.where(Domain_Test_2s >= 3, 1, 0)
# Binarize Like_Train_2s with threshold 3
Domain_Train_2s_binary = np.where(Domain_Train_2s >= 3, 1, 0)

In [7]:
svmrbf_arousal_2s_psds = SVC(kernel='rbf',max_iter = 25000,random_state=42)
svmlinear_arousal_2s_psds = SVC(kernel='linear',max_iter=25000,random_state=42)

# Train the models on the GPU
svmrbf_arousal_2s_psds.fit(X_2s, Arousal_Train_2s_binary)
svmlinear_arousal_2s_psds.fit(X_2s, Arousal_Train_2s_binary)

# Predict on new data
predictions_rbf = svmrbf_arousal_2s_psds.predict(M_2s)
predictions_linear = svmlinear_arousal_2s_psds.predict(M_2s)

# Evaluate the models
svmrbf_arousal_2s_psds_accuracy = accuracy_score(Arousal_Test_2s_binary, predictions_rbf)
svmrbf_arousal_2s_psds_f1 = f1_score(Arousal_Test_2s_binary, predictions_rbf, average='weighted')

svmlinear_arousal_2s_psds_accuracy = accuracy_score(Arousal_Test_2s_binary, predictions_linear)
svmlinear_arousal_2s_psds_f1 = f1_score(Arousal_Test_2s_binary, predictions_linear, average='weighted')

print("RBF SVM:")
print(f"Accuracy: {svmrbf_arousal_2s_psds_accuracy * 100:.2f}%")
print(f"F1 Score: {svmrbf_arousal_2s_psds_f1:.2f}")

print("Linear SVM:")
print(f"Accuracy: {svmlinear_arousal_2s_psds_accuracy * 100:.2f}%")
print(f"F1 Score: {svmlinear_arousal_2s_psds_f1:.2f}")


/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


RBF SVM:
Accuracy: 68.08%
F1 Score: 0.64
Linear SVM:
Accuracy: 62.59%
F1 Score: 0.53


In [8]:
svmrbf_valence_2s_psds = SVC(kernel='rbf',max_iter = 25000,random_state=42)
svmlinear_valence_2s_psds = SVC(kernel='linear',max_iter = 25000,random_state=42)

# Train the models on the GPU
svmrbf_valence_2s_psds.fit(X_2s, Valence_Train_2s_binary)
svmlinear_valence_2s_psds.fit(X_2s, Valence_Train_2s_binary)

# Predict on new data
predictions_rbf = svmrbf_valence_2s_psds.predict(M_2s)
predictions_linear = svmlinear_valence_2s_psds.predict(M_2s)

# Evaluate the models
svmrbf_valence_2s_psds_accuracy = accuracy_score(Valence_Test_2s_binary, predictions_rbf)
svmrbf_valence_2s_psds_f1 = f1_score(Valence_Test_2s_binary, predictions_rbf, average='weighted')

svmlinear_valence_2s_psds_accuracy = accuracy_score(Valence_Test_2s_binary, predictions_linear)
svmlinear_valence_2s_psds_f1 = f1_score(Valence_Test_2s_binary, predictions_linear, average='weighted')

print("RBF SVM:")
print(f"Accuracy: {svmrbf_valence_2s_psds_accuracy * 100:.2f}%")
print(f"F1 Score: {svmrbf_valence_2s_psds_f1:.2f}")

print("Linear SVM:")
print(f"Accuracy: {svmlinear_valence_2s_psds_accuracy * 100:.2f}%")
print(f"F1 Score: {svmlinear_valence_2s_psds_f1:.2f}")

/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


RBF SVM:
Accuracy: 79.60%
F1 Score: 0.74
Linear SVM:
Accuracy: 77.57%
F1 Score: 0.68


In [9]:
# Initialize the SVM models with RBF and linear kernels
svmrbf_domain_2s_psds = SVC(kernel='rbf',max_iter = 25000,random_state=42)
svmlinear_domain_2s_psds = SVC(kernel='linear',max_iter = 25000,random_state=42)

# Train the models on the GPU
svmrbf_domain_2s_psds.fit(X_2s, Domain_Train_2s_binary)
svmlinear_domain_2s_psds.fit(X_2s, Domain_Train_2s_binary)

# Predict on new data
predictions_rbf = svmrbf_domain_2s_psds.predict(M_2s)
predictions_linear = svmlinear_domain_2s_psds.predict(M_2s)

# Evaluate the models
svmrbf_domain_2s_psds_accuracy = accuracy_score(Domain_Test_2s_binary, predictions_rbf)
svmrbf_domain_2s_psds_f1 = f1_score(Domain_Test_2s_binary, predictions_rbf, average='weighted')

svmlinear_domain_2s_psds_accuracy = accuracy_score(Domain_Test_2s_binary, predictions_linear)
svmlinear_domain_2s_psds_f1 = f1_score(Domain_Test_2s_binary, predictions_linear, average='weighted')

print("RBF SVM:")
print(f"Accuracy: {svmrbf_domain_2s_psds_accuracy * 100:.2f}%")
print(f"F1 Score: {svmrbf_domain_2s_psds_f1:.2f}")

print("Linear SVM:")
print(f"Accuracy: {svmlinear_domain_2s_psds_accuracy * 100:.2f}%")
print(f"F1 Score: {svmlinear_domain_2s_psds_f1:.2f}")

/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


RBF SVM:
Accuracy: 82.88%
F1 Score: 0.78
Linear SVM:
Accuracy: 80.94%
F1 Score: 0.73


In [10]:
from enum import Enum
class Norm(Enum):
    Manhattan = 1
    Euclidean = 2
n     = 5
norm  = Norm.Euclidean.value
from sklearn import neighbors, linear_model
from sklearn.metrics import classification_report
knn = neighbors.KNeighborsClassifier(n_neighbors=n, weights='uniform', p=norm)

In [11]:
print(int(math.sqrt(len(X_2s))))


260


In [12]:
n = int(math.sqrt(len(X_2s)))
knn = neighbors.KNeighborsClassifier(n_neighbors=n, weights='uniform', p=norm)
knn_arousal_psds = knn.fit(X_2s, Arousal_Train_2s_binary)
print('k_NN accuracy for test set: %f' % knn_arousal_psds.score(M_2s, Arousal_Test_2s_binary))
y_true, y_pred = Arousal_Test_2s_binary, knn_arousal_psds.predict(M_2s)
print(classification_report(y_true, y_pred))

k_NN accuracy for test set: 0.742346
              precision    recall  f1-score   support

           0       0.74      0.53      0.62      6678
           1       0.74      0.88      0.81     10306

    accuracy                           0.74     16984
   macro avg       0.74      0.70      0.71     16984
weighted avg       0.74      0.74      0.73     16984



In [13]:

knn_valence_psds = knn.fit(X_2s, Valence_Train_2s_binary)
print('k_NN accuracy for test set: %f' % knn_valence_psds.score(M_2s, Valence_Test_2s_binary))
y_true, y_pred = Valence_Test_2s_binary, knn_valence_psds.predict(M_2s)
print(classification_report(y_true, y_pred))

k_NN accuracy for test set: 0.809585
              precision    recall  f1-score   support

           0       0.73      0.28      0.40      3907
           1       0.82      0.97      0.89     13077

    accuracy                           0.81     16984
   macro avg       0.77      0.62      0.64     16984
weighted avg       0.80      0.81      0.77     16984



In [14]:
knn_domain_psds = knn.fit(X_2s, Domain_Train_2s_binary)
print('k_NN accuracy for test set: %f' % knn_domain_psds.score(M_2s, Domain_Test_2s_binary))
y_true, y_pred = Domain_Test_2s_binary, knn_domain_psds.predict(M_2s)
print(classification_report(y_true, y_pred))

k_NN accuracy for test set: 0.832607
              precision    recall  f1-score   support

           0       0.69      0.26      0.38      3315
           1       0.84      0.97      0.90     13669

    accuracy                           0.83     16984
   macro avg       0.77      0.62      0.64     16984
weighted avg       0.81      0.83      0.80     16984



In [15]:
Val_R_2s = RandomForestRegressor(n_estimators=512, n_jobs=6,random_state=42)
Val_R_2s.fit(X_2s, Valence_Train_2s_binary)
Valence_pred_2s = Val_R_2s.predict(M_2s)


In [16]:
Valence_pred_2s = np.where(Valence_pred_2s >= 0.5, 1, 0)

In [17]:
Ran_valence_2s_psds_accuracy = accuracy_score(Valence_Test_2s_binary, Valence_pred_2s)
Ran_valence_2s_psds_f1 = f1_score(Valence_Test_2s_binary, Valence_pred_2s, average='weighted')

In [18]:
print(Ran_valence_2s_psds_accuracy,Ran_valence_2s_psds_f1)

0.9070890249646726 0.9011202454448259


In [19]:
Aro_R_2s = RandomForestRegressor(n_estimators=512, n_jobs=6,random_state=42)
Aro_R_2s.fit(X_2s, Arousal_Train_2s_binary)
Arousal_pred_2s = Aro_R_2s.predict(M_2s)

Arousal_pred_2s = np.where(Arousal_pred_2s >= 0.5, 1, 0)
Ran_Arousal_2s_psds_accuracy = accuracy_score(Arousal_Test_2s_binary, Arousal_pred_2s)
Ran_Arousal_2s_psds_f1 = f1_score(Arousal_Test_2s_binary, Arousal_pred_2s, average='weighted')

print(Ran_Arousal_2s_psds_accuracy,Ran_Arousal_2s_psds_f1)

0.8764130946773434 0.8745463309683013


In [20]:
Dom_R_2s = RandomForestRegressor(n_estimators=512, n_jobs=6,random_state=42)
Dom_R_2s.fit(X_2s, Domain_Train_2s_binary)
Domain_pred_2s = Dom_R_2s.predict(M_2s)

Domain_pred_2s = np.where(Domain_pred_2s >= 0.5, 1, 0)
Ran_Domain_2s_psds_accuracy = accuracy_score(Domain_Test_2s_binary, Domain_pred_2s)
Ran_Domain_2s_psds_f1 = f1_score(Domain_Test_2s_binary, Domain_pred_2s, average='weighted')

print(Ran_Domain_2s_psds_accuracy,Ran_Domain_2s_psds_f1)

0.9166862929816297 0.9102911998845671


In [22]:
with open('out_ica_data_training_2s.npy', 'rb') as fileTrain:
    X_2s_ica  = np.load(fileTrain)
    
with open('out_ica_label_training_2s.npy', 'rb') as fileTrainL:
    Y_2s_ica  = np.load(fileTrainL)

ss = StandardScaler()
X_2s_ica = ss.fit_transform(X_2s_ica)
Z_2s_ica = np.ravel(Y_2s_ica[:, [1]])

Arousal_Train_ica = np.ravel(Y_2s_ica[:, [0]])
Valence_Train_ica = np.ravel(Y_2s_ica[:, [1]])
Domain_Train_ica = np.ravel(Y_2s_ica[:, [2]])




with open('out_ica_data_testing_2s.npy', 'rb') as fileTrain:
    M_2s_ica  = np.load(fileTrain)
    
with open('out_ica_label_testing_2s.npy', 'rb') as fileTrainL:
    N_2s_ica  = np.load(fileTrainL)

M_2s_ica = ss.transform(M_2s_ica)
L_2s_ica = np.ravel(N_2s_ica[:, [1]])

Arousal_Test_ica = np.ravel(N_2s_ica[:, [0]])
Valence_Test_ica = np.ravel(N_2s_ica[:, [1]])
Domain_Test_ica = np.ravel(N_2s_ica[:, [2]])



Arousal_Test_ica_binary = np.where(Arousal_Test_ica >= 3, 1, 0)
Arousal_Train_ica_binary = np.where(Arousal_Train_ica >= 3, 1, 0)

Valence_Train_ica_binary = np.where(Valence_Train_ica >= 3, 1, 0)

Valence_Test_ica_binary = np.where(Valence_Test_ica >= 3, 1, 0)

Domain_Test_ica_binary = np.where(Domain_Test_ica >= 3, 1, 0)

Domain_Train_ica_binary = np.where(Domain_Train_ica >= 3, 1, 0)

In [23]:
svmlinear_arousal_2s_ica = SVC(kernel='linear',max_iter=25000,random_state=42,verbose=True)
svmlinear_arousal_2s_ica.fit(X_2s_ica, Arousal_Train_ica_binary)
predictions_linear_arousal = svmlinear_arousal_2s_ica.predict(M_2s_ica)


svmlinear_arousal_2s_ica_accuracy = accuracy_score(Arousal_Test_ica_binary, predictions_linear_arousal)
svmlinear_arousal_2s_ica_f1 = f1_score(Arousal_Test_ica_binary, predictions_linear_arousal, average='weighted')
print("Arousal Linear SVM:")
print(f"Accuracy: {svmlinear_arousal_2s_ica_accuracy * 100:.2f}%")
print(f"F1 Score: {svmlinear_arousal_2s_ica_f1:.2f}")


[LibSVM]........................WARN: libsvm Solver reached max_iter
optimization finished, #iter = 25000
obj = -1939.278313, rho = 0.687785
nSV = 5430, nBSV = 894
Total nSV = 5430


/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=25000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Arousal Linear SVM:
Accuracy: 59.64%
F1 Score: 0.50


In [24]:
svmrbf_arousal_2s_ica = SVC(kernel='rbf',max_iter=25000,random_state=42,verbose=True)

# Train the models
svmrbf_arousal_2s_ica.fit(X_2s_ica, Arousal_Train_ica_binary)

# Predict on new data
predictions_rbf_arousal = svmrbf_arousal_2s_ica.predict(M_2s_ica)


# Evaluate the models
svmrbf_arousal_2s_ica_accuracy = accuracy_score(Arousal_Test_ica_binary, predictions_rbf_arousal)
svmrbf_arousal_2s_ica_f1 = f1_score(Arousal_Test_ica_binary, predictions_rbf_arousal, average='weighted')



# Print the results
print("Arousal RBF SVM:")
print(f"Accuracy: {svmrbf_arousal_2s_ica_accuracy * 100:.2f}%")
print(f"F1 Score: {svmrbf_arousal_2s_ica_f1:.2f}")




[LibSVM]........................WARN: libsvm Solver reached max_iter
optimization finished, #iter = 25000
obj = -24214.631120, rho = 0.293178
nSV = 29627, nBSV = 26180
Total nSV = 29627


/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=25000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Arousal RBF SVM:
Accuracy: 92.73%
F1 Score: 0.93


In [25]:
svmrbf_valence_2s_ica = SVC(kernel='rbf',max_iter=25000,random_state=42,verbose=True)
svmlinear_valence_2s_ica = SVC(kernel='linear',max_iter=25000,random_state=42,verbose=True)

svmrbf_valence_2s_ica.fit(X_2s_ica, Valence_Train_ica_binary)
svmlinear_valence_2s_ica.fit(X_2s_ica, Valence_Train_ica_binary)

predictions_rbf_valence = svmrbf_valence_2s_ica.predict(M_2s_ica)
predictions_linear_valence = svmlinear_valence_2s_ica.predict(M_2s_ica)

svmrbf_valence_2s_ica_accuracy = accuracy_score(Valence_Test_ica_binary, predictions_rbf_valence)
svmrbf_valence_2s_ica_f1 = f1_score(Valence_Test_ica_binary, predictions_rbf_valence, average='weighted')

svmlinear_valence_2s_ica_accuracy = accuracy_score(Valence_Test_ica_binary, predictions_linear_valence)
svmlinear_valence_2s_ica_f1 = f1_score(Valence_Test_ica_binary, predictions_linear_valence, average='weighted')

print("Valence RBF SVM:")
print(f"Accuracy: {svmrbf_valence_2s_ica_accuracy * 100:.2f}%")
print(f"F1 Score: {svmrbf_valence_2s_ica_f1:.2f}")

print("Valence Linear SVM:")
print(f"Accuracy: {svmlinear_valence_2s_ica_accuracy * 100:.2f}%")
print(f"F1 Score: {svmlinear_valence_2s_ica_f1:.2f}")


[LibSVM]........................WARN: libsvm Solver reached max_iter
optimization finished, #iter = 25000
obj = -18695.693570, rho = 0.637128
nSV = 23014, nBSV = 19867
Total nSV = 23014
[LibSVM]

/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=25000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


........................WARN: libsvm Solver reached max_iter
optimization finished, #iter = 25000
obj = -1979.588488, rho = 1.155865
nSV = 6394, nBSV = 797
Total nSV = 6394


/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=25000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Valence RBF SVM:
Accuracy: 93.96%
F1 Score: 0.94
Valence Linear SVM:
Accuracy: 74.51%
F1 Score: 0.69


In [26]:
svmrbf_domain_2s_ica = SVC(kernel='rbf',max_iter=25000,random_state=42,verbose=True)
svmlinear_domain_2s_ica = SVC(kernel='linear',max_iter=25000,random_state=42,verbose=True)

svmrbf_domain_2s_ica.fit(X_2s_ica, Domain_Train_ica_binary)

svmlinear_domain_2s_ica.fit(X_2s_ica, Domain_Train_ica_binary)

predictions_rbf_domain = svmrbf_domain_2s_ica.predict(M_2s_ica)
predictions_linear_domain = svmlinear_domain_2s_ica.predict(M_2s_ica)

svmrbf_domain_2s_ica_accuracy = accuracy_score(Domain_Test_ica_binary, predictions_rbf_domain)
svmrbf_domain_2s_ica_f1 = f1_score(Domain_Test_ica_binary, predictions_rbf_domain, average='weighted')

svmlinear_domain_2s_ica_accuracy = accuracy_score(Domain_Test_ica_binary, predictions_linear_domain)
svmlinear_domain_2s_ica_f1 = f1_score(Domain_Test_ica_binary, predictions_linear_domain, average='weighted')

print("Domain RBF SVM:")
print(f"Accuracy: {svmrbf_domain_2s_ica_accuracy * 100:.2f}%")
print(f"F1 Score: {svmrbf_domain_2s_ica_f1:.2f}")

print("Domain Linear SVM:")
print(f"Accuracy: {svmlinear_domain_2s_ica_accuracy * 100:.2f}%")
print(f"F1 Score: {svmlinear_domain_2s_ica_f1:.2f}")

[LibSVM].....................*...WARN: libsvm Solver reached max_iter
optimization finished, #iter = 25000
obj = -13558.232844, rho = 0.750782
nSV = 19740, nBSV = 16699
Total nSV = 19740
[LibSVM]

/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=25000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


........................WARN: libsvm Solver reached max_iter
optimization finished, #iter = 25000
obj = -2675.878007, rho = 0.567135
nSV = 7975, nBSV = 1215
Total nSV = 7975


/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=25000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Domain RBF SVM:
Accuracy: 95.31%
F1 Score: 0.95
Domain Linear SVM:
Accuracy: 76.81%
F1 Score: 0.72


In [27]:
n = int(math.sqrt(len(X_2s_ica)))
knn = neighbors.KNeighborsClassifier(n_neighbors=n, weights='uniform', p=norm)
knn_arousal_ica = knn.fit(X_2s_ica, Arousal_Train_ica_binary)
print('k_NN accuracy for test set: %f' % knn_arousal_ica.score(M_2s_ica, Arousal_Test_ica_binary))
y_true, y_pred = Arousal_Test_ica_binary, knn_arousal_ica.predict(M_2s_ica)
print(classification_report(y_true, y_pred))

knn_valence_ica = knn.fit(X_2s_ica, Valence_Train_ica_binary)
print('k_NN accuracy for test set: %f' % knn_valence_ica.score(M_2s_ica, Valence_Test_ica_binary))
y_true, y_pred = Valence_Test_ica_binary, knn_valence_ica.predict(M_2s_ica)
print(classification_report(y_true, y_pred))

knn_domain_ica = knn.fit(X_2s_ica, Domain_Train_ica_binary)
print('k_NN accuracy for test set: %f' % knn_domain_ica.score(M_2s_ica, Domain_Test_ica_binary))
y_true, y_pred = Domain_Test_ica_binary, knn_domain_ica.predict(M_2s_ica)
print(classification_report(y_true, y_pred))

k_NN accuracy for test set: 0.904145
              precision    recall  f1-score   support

           0       0.93      0.82      0.87      6678
           1       0.89      0.96      0.92     10306

    accuracy                           0.90     16984
   macro avg       0.91      0.89      0.90     16984
weighted avg       0.91      0.90      0.90     16984

k_NN accuracy for test set: 0.896844
              precision    recall  f1-score   support

           0       0.97      0.57      0.72      3907
           1       0.89      0.99      0.94     13077

    accuracy                           0.90     16984
   macro avg       0.93      0.78      0.83     16984
weighted avg       0.90      0.90      0.89     16984

k_NN accuracy for test set: 0.904322
              precision    recall  f1-score   support

           0       0.97      0.52      0.68      3315
           1       0.90      1.00      0.94     13669

    accuracy                           0.90     16984
   macro avg     

In [28]:
knn_domain_ica = knn.fit(X_2s_ica, Domain_Train_ica_binary)
print('k_NN accuracy for test set: %f' % knn_domain_ica.score(M_2s_ica, Domain_Test_ica_binary))
y_true, y_pred = Domain_Test_ica_binary, knn_domain_ica.predict(M_2s_ica)
print(classification_report(y_true, y_pred))

k_NN accuracy for test set: 0.904322
              precision    recall  f1-score   support

           0       0.97      0.52      0.68      3315
           1       0.90      1.00      0.94     13669

    accuracy                           0.90     16984
   macro avg       0.94      0.76      0.81     16984
weighted avg       0.91      0.90      0.89     16984



In [29]:
Val_R_ica = RandomForestRegressor(n_estimators=512, n_jobs=6,random_state=42)
Val_R_ica.fit(X_2s_ica, Valence_Train_ica_binary)
Valence_pred_ica = Val_R_ica.predict(M_2s_ica)
Valence_pred_ica = np.where(Valence_pred_ica >= 0.5, 1, 0)
Ran_valence_ica_accuracy = accuracy_score(Valence_Test_ica_binary, Valence_pred_ica)
Ran_valence_ica_f1 = f1_score(Valence_Test_ica_binary, Valence_pred_ica, average='weighted')
print(Ran_valence_ica_accuracy,Ran_valence_ica_f1)

Aro_R_ica = RandomForestRegressor(n_estimators=512, n_jobs=6,random_state=42)
Aro_R_ica.fit(X_2s_ica, Arousal_Train_ica_binary)
Arousal_pred_ica = Aro_R_ica.predict(M_2s_ica)

Arousal_pred_ica= np.where(Arousal_pred_ica >= 0.5, 1, 0)
Ran_Arousal_ica_accuracy = accuracy_score(Arousal_Test_ica_binary, Arousal_pred_ica)
Ran_Arousal_ica_f1 = f1_score(Arousal_Test_ica_binary, Arousal_pred_ica, average='weighted')

print(Ran_Arousal_ica_accuracy,Ran_Arousal_ica_f1)

Dom_R_ica = RandomForestRegressor(n_estimators=512, n_jobs=6,random_state=42)
Dom_R_ica.fit(X_2s_ica, Domain_Train_ica_binary)
Domain_pred_ica = Dom_R_ica.predict(M_2s_ica)

Domain_pred_ica = np.where(Domain_pred_ica >= 0.5, 1, 0)
Ran_Domain_ica_accuracy = accuracy_score(Domain_Test_ica_binary, Domain_pred_ica)
Ran_Domain_ica_f1 = f1_score(Domain_Test_ica_binary, Domain_pred_ica, average='weighted')

print(Ran_Domain_ica_accuracy,Ran_Domain_ica_f1)

0.962670748940179 0.9615876188284762
0.9615520489872822 0.9613550833503112
0.9617875647668394 0.9602883268661352


In [30]:
with open('out_dwt_data_training_2s.npy', 'rb') as fileTrain:
    X_2s_dwt  = np.load(fileTrain)
    
with open('out_dwt_label_training_2s.npy', 'rb') as fileTrainL:
    Y_2s_dwt  = np.load(fileTrainL)

ss = StandardScaler()
X_2s_dwt = ss.fit_transform(X_2s_dwt)
Z_2s_dwt = np.ravel(Y_2s_dwt[:, [1]])

Arousal_Train_dwt = np.ravel(Y_2s_dwt[:, [0]])
Valence_Train_dwt = np.ravel(Y_2s_dwt[:, [1]])
Domain_Train_dwt = np.ravel(Y_2s_dwt[:, [2]])




with open('out_dwt_data_testing_2s.npy', 'rb') as fileTrain:
    M_2s_dwt  = np.load(fileTrain)
    
with open('out_dwt_label_testing_2s.npy', 'rb') as fileTrainL:
    N_2s_dwt  = np.load(fileTrainL)

M_2s_dwt = ss.transform(M_2s_dwt)
L_2s_dwt = np.ravel(N_2s_dwt[:, [1]])

Arousal_Test_dwt = np.ravel(N_2s_dwt[:, [0]])
Valence_Test_dwt = np.ravel(N_2s_dwt[:, [1]])
Domain_Test_dwt = np.ravel(N_2s_dwt[:, [2]])



Arousal_Test_dwt_binary = np.where(Arousal_Test_dwt >= 3, 1, 0)
Arousal_Train_dwt_binary = np.where(Arousal_Train_dwt >= 3, 1, 0)

Valence_Train_dwt_binary = np.where(Valence_Train_dwt >= 3, 1, 0)

Valence_Test_dwt_binary = np.where(Valence_Test_dwt >= 3, 1, 0)

Domain_Test_dwt_binary = np.where(Domain_Test_dwt >= 3, 1, 0)

Domain_Train_dwt_binary = np.where(Domain_Train_dwt >= 3, 1, 0)


In [31]:
svmlinear_arousal_2s_dwt = SVC(kernel='linear',max_iter=25000,random_state=42,verbose=True)
svmlinear_arousal_2s_dwt.fit(X_2s_dwt, Arousal_Train_dwt_binary)
predictions_linear_arousal = svmlinear_arousal_2s_dwt.predict(M_2s_dwt)


svmlinear_arousal_2s_dwt_accuracy = accuracy_score(Arousal_Test_dwt_binary, predictions_linear_arousal)
svmlinear_arousal_2s_dwt_f1 = f1_score(Arousal_Test_dwt_binary, predictions_linear_arousal, average='weighted')
print("Arousal Linear SVM:")
print(f"Accuracy: {svmlinear_arousal_2s_dwt_accuracy * 100:.2f}%")
print(f"F1 Score: {svmlinear_arousal_2s_dwt_f1:.2f}")
svmrbf_arousal_2s_dwt = SVC(kernel='rbf',max_iter=25000,random_state=42,verbose=True)

# Train the models
svmrbf_arousal_2s_dwt.fit(X_2s_dwt, Arousal_Train_dwt_binary)

# Predict on new data
predictions_rbf_arousal = svmrbf_arousal_2s_dwt.predict(M_2s_dwt)


# Evaluate the models
svmrbf_arousal_2s_dwt_accuracy = accuracy_score(Arousal_Test_dwt_binary, predictions_rbf_arousal)
svmrbf_arousal_2s_dwt_f1 = f1_score(Arousal_Test_dwt_binary, predictions_rbf_arousal, average='weighted')



# Print the results
print("Arousal RBF SVM:")
print(f"Accuracy: {svmrbf_arousal_2s_dwt_accuracy * 100:.2f}%")
print(f"F1 Score: {svmrbf_arousal_2s_dwt_f1:.2f}")

[LibSVM]........................WARN: libsvm Solver reached max_iter
optimization finished, #iter = 25000
obj = -45973.549097, rho = 12.390290
nSV = 45303, nBSV = 44914
Total nSV = 45303


/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=25000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Arousal Linear SVM:
Accuracy: 60.36%
F1 Score: 0.46
[LibSVM]........................WARN: libsvm Solver reached max_iter
optimization finished, #iter = 25000
obj = -46891.814596, rho = 0.107668
nSV = 49992, nBSV = 49992
Total nSV = 49992


/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=25000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Arousal RBF SVM:
Accuracy: 61.94%
F1 Score: 0.49


In [32]:
svmrbf_valence_2s_dwt = SVC(kernel='rbf',max_iter=25000,random_state=42,verbose=True)
svmlinear_valence_2s_dwt = SVC(kernel='linear',max_iter=25000,random_state=42,verbose=True)

svmrbf_valence_2s_dwt.fit(X_2s_dwt, Valence_Train_dwt_binary)
svmlinear_valence_2s_dwt.fit(X_2s_dwt, Valence_Train_dwt_binary)

predictions_rbf_valence = svmrbf_valence_2s_dwt.predict(M_2s_dwt)
predictions_linear_valence = svmlinear_valence_2s_dwt.predict(M_2s_dwt)

svmrbf_valence_2s_dwt_accuracy = accuracy_score(Valence_Test_dwt_binary, predictions_rbf_valence)
svmrbf_valence_2s_dwt_f1 = f1_score(Valence_Test_dwt_binary, predictions_rbf_valence, average='weighted')

svmlinear_valence_2s_dwt_accuracy = accuracy_score(Valence_Test_dwt_binary, predictions_linear_valence)
svmlinear_valence_2s_dwt_f1 = f1_score(Valence_Test_dwt_binary, predictions_linear_valence, average='weighted')

print("Valence RBF SVM:")
print(f"Accuracy: {svmrbf_valence_2s_dwt_accuracy * 100:.2f}%")
print(f"F1 Score: {svmrbf_valence_2s_dwt_f1:.2f}")

print("Valence Linear SVM:")
print(f"Accuracy: {svmlinear_valence_2s_dwt_accuracy * 100:.2f}%")
print(f"F1 Score: {svmlinear_valence_2s_dwt_f1:.2f}")

[LibSVM]........................WARN: libsvm Solver reached max_iter
optimization finished, #iter = 25000
obj = -27585.619871, rho = 0.380767
nSV = 30702, nBSV = 30392
Total nSV = 30702
[LibSVM]

/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=25000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


........................WARN: libsvm Solver reached max_iter
optimization finished, #iter = 25000
obj = -34791.504948, rho = 1.591641
nSV = 30906, nBSV = 30405
Total nSV = 30906


/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=25000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Valence RBF SVM:
Accuracy: 78.27%
F1 Score: 0.70
Valence Linear SVM:
Accuracy: 77.20%
F1 Score: 0.68


In [33]:
svmrbf_domain_2s_dwt = SVC(kernel='rbf',max_iter=25000,random_state=42,verbose=True)
svmlinear_domain_2s_dwt = SVC(kernel='linear',max_iter=25000,random_state=42,verbose=True)

svmrbf_domain_2s_dwt.fit(X_2s_dwt, Domain_Train_dwt_binary)

svmlinear_domain_2s_dwt.fit(X_2s_dwt, Domain_Train_dwt_binary)

predictions_rbf_domain = svmrbf_domain_2s_dwt.predict(M_2s_dwt)
predictions_linear_domain = svmlinear_domain_2s_dwt.predict(M_2s_dwt)

svmrbf_domain_2s_dwt_accuracy = accuracy_score(Domain_Test_dwt_binary, predictions_rbf_domain)
svmrbf_domain_2s_dwt_f1 = f1_score(Domain_Test_dwt_binary, predictions_rbf_domain, average='weighted')

svmlinear_domain_2s_dwt_accuracy = accuracy_score(Domain_Test_dwt_binary, predictions_linear_domain)
svmlinear_domain_2s_dwt_f1 = f1_score(Domain_Test_dwt_binary, predictions_linear_domain, average='weighted')

print("Domain RBF SVM:")
print(f"Accuracy: {svmrbf_domain_2s_dwt_accuracy * 100:.2f}%")
print(f"F1 Score: {svmrbf_domain_2s_dwt_f1:.2f}")

print("Domain Linear SVM:")
print(f"Accuracy: {svmlinear_domain_2s_dwt_accuracy * 100:.2f}%")
print(f"F1 Score: {svmlinear_domain_2s_dwt_f1:.2f}")

[LibSVM]........................WARN: libsvm Solver reached max_iter
optimization finished, #iter = 25000
obj = -26339.688997, rho = 0.683314
nSV = 26965, nBSV = 26601
Total nSV = 26965
[LibSVM]

/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=25000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


........................WARN: libsvm Solver reached max_iter
optimization finished, #iter = 25000
obj = -31312.079626, rho = 1.409417
nSV = 26828, nBSV = 26535
Total nSV = 26828


/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=25000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Domain RBF SVM:
Accuracy: 81.19%
F1 Score: 0.74
Domain Linear SVM:
Accuracy: 81.06%
F1 Score: 0.73


In [34]:
n = int(math.sqrt(len(X_2s_dwt)))
knn = neighbors.KNeighborsClassifier(n_neighbors=n, weights='uniform', p=norm)
knn_arousal_dwt = knn.fit(X_2s_dwt, Arousal_Train_dwt_binary)
print('k_NN accuracy for test set: %f' % knn_arousal_dwt.score(M_2s_dwt, Arousal_Test_dwt_binary))
y_true, y_pred = Arousal_Test_dwt_binary, knn_arousal_dwt.predict(M_2s_dwt)
print(classification_report(y_true, y_pred))

knn_valence_dwt = knn.fit(X_2s_dwt, Valence_Train_dwt_binary)
print('k_NN accuracy for test set: %f' % knn_valence_dwt.score(M_2s_dwt, Valence_Test_dwt_binary))
y_true, y_pred = Valence_Test_dwt_binary, knn_valence_dwt.predict(M_2s_dwt)
print(classification_report(y_true, y_pred))

knn_domain_dwt = knn.fit(X_2s_dwt, Domain_Train_dwt_binary)
print('k_NN accuracy for test set: %f' % knn_domain_dwt.score(M_2s_dwt, Domain_Test_dwt_binary))
y_true, y_pred = Domain_Test_dwt_binary, knn_domain_dwt.predict(M_2s_dwt)
print(classification_report(y_true, y_pred))

k_NN accuracy for test set: 0.686293
              precision    recall  f1-score   support

           0       0.68      0.38      0.48      6678
           1       0.69      0.89      0.77     10306

    accuracy                           0.69     16984
   macro avg       0.69      0.63      0.63     16984
weighted avg       0.69      0.69      0.66     16984

k_NN accuracy for test set: 0.792864
              precision    recall  f1-score   support

           0       0.72      0.16      0.26      3907
           1       0.80      0.98      0.88     13077

    accuracy                           0.79     16984
   macro avg       0.76      0.57      0.57     16984
weighted avg       0.78      0.79      0.74     16984

k_NN accuracy for test set: 0.821244
              precision    recall  f1-score   support

           0       0.69      0.15      0.25      3315
           1       0.83      0.98      0.90     13669

    accuracy                           0.82     16984
   macro avg     

In [35]:
Val_R_dwt = RandomForestRegressor(n_estimators=512,random_state=42, n_jobs=6)
Val_R_dwt.fit(X_2s_dwt, Valence_Train_dwt_binary)
Valence_pred_dwt = Val_R_dwt.predict(M_2s_dwt)
Valence_pred_dwt = np.where(Valence_pred_dwt >= 0.5, 1, 0)
Ran_valence_dwt_accuracy = accuracy_score(Valence_Test_dwt_binary, Valence_pred_dwt)
Ran_valence_dwt_f1 = f1_score(Valence_Test_dwt_binary, Valence_pred_dwt, average='weighted')
print(Ran_valence_dwt_accuracy,Ran_valence_dwt_f1)

Aro_R_dwt = RandomForestRegressor(n_estimators=512,random_state=42, n_jobs=6)
Aro_R_dwt.fit(X_2s_dwt, Arousal_Train_dwt_binary)
Arousal_pred_dwt = Aro_R_dwt.predict(M_2s_dwt)

Arousal_pred_dwt= np.where(Arousal_pred_dwt >= 0.5, 1, 0)
Ran_Arousal_dwt_accuracy = accuracy_score(Arousal_Test_dwt_binary, Arousal_pred_dwt)
Ran_Arousal_dwt_f1 = f1_score(Arousal_Test_dwt_binary, Arousal_pred_dwt, average='weighted')

print(Ran_Arousal_dwt_accuracy,Ran_Arousal_dwt_f1)

Dom_R_dwt = RandomForestRegressor(n_estimators=512,random_state=42, n_jobs=6)
Dom_R_dwt.fit(X_2s_dwt, Domain_Train_dwt_binary)
Domain_pred_dwt = Dom_R_dwt.predict(M_2s_dwt)

Domain_pred_dwt = np.where(Domain_pred_dwt >= 0.5, 1, 0)


0.898551577955723 0.8913906584929366
0.8615167216203485 0.8591526161209249


In [36]:
Ran_Domain_dwt_accuracy = accuracy_score(Domain_Test_dwt_binary, Domain_pred_dwt)
Ran_Domain_dwt_f1 = f1_score(Domain_Test_dwt_binary, Domain_pred_dwt, average='weighted')

print(Ran_Domain_dwt_accuracy,Ran_Domain_dwt_f1)

0.9077955723033443 0.8997912004737952


In [37]:
with open('out_dwt_ica_data_training_2s.npy', 'rb') as fileTrain:
    X_2s_dwt_ica  = np.load(fileTrain)
    
with open('out_dwt_ica_label_training_2s.npy', 'rb') as fileTrainL:
    Y_2s_dwt_ica  = np.load(fileTrainL)

ss = StandardScaler()
X_2s_dwt_ica = ss.fit_transform(X_2s_dwt_ica)
Z_2s_dwt_ica = np.ravel(Y_2s_dwt_ica[:, [1]])

Arousal_Train_dwt_ica = np.ravel(Y_2s_dwt_ica[:, [0]])
Valence_Train_dwt_ica = np.ravel(Y_2s_dwt_ica[:, [1]])
Domain_Train_dwt_ica = np.ravel(Y_2s_dwt_ica[:, [2]])




with open('out_dwt_ica_data_testing_2s.npy', 'rb') as fileTrain:
    M_2s_dwt_ica  = np.load(fileTrain)
    
with open('out_dwt_ica_label_testing_2s.npy', 'rb') as fileTrainL:
    N_2s_dwt_ica  = np.load(fileTrainL)

M_2s_dwt_ica = ss.transform(M_2s_dwt_ica)
L_2s_dwt_ica = np.ravel(N_2s_dwt_ica[:, [1]])

Arousal_Test_dwt_ica = np.ravel(N_2s_dwt_ica[:, [0]])
Valence_Test_dwt_ica = np.ravel(N_2s_dwt_ica[:, [1]])
Domain_Test_dwt_ica = np.ravel(N_2s_dwt_ica[:, [2]])



Arousal_Test_dwt_ica_binary = np.where(Arousal_Test_dwt_ica >= 3, 1, 0)
Arousal_Train_dwt_ica_binary = np.where(Arousal_Train_dwt_ica >= 3, 1, 0)

Valence_Train_dwt_ica_binary = np.where(Valence_Train_dwt_ica >= 3, 1, 0)

Valence_Test_dwt_ica_binary = np.where(Valence_Test_dwt_ica >= 3, 1, 0)

Domain_Test_dwt_ica_binary = np.where(Domain_Test_dwt_ica >= 3, 1, 0)

Domain_Train_dwt_ica_binary = np.where(Domain_Train_dwt_ica >= 3, 1, 0)


In [38]:
svmlinear_arousal_2s_dwt_ica = SVC(kernel='linear',max_iter=25000,random_state=42,verbose=True)
svmlinear_arousal_2s_dwt_ica.fit(X_2s_dwt_ica, Arousal_Train_dwt_ica_binary)
predictions_linear_arousal = svmlinear_arousal_2s_dwt_ica.predict(M_2s_dwt_ica)


svmlinear_arousal_2s_dwt_ica_accuracy = accuracy_score(Arousal_Test_dwt_ica_binary, predictions_linear_arousal)
svmlinear_arousal_2s_dwt_ica_f1 = f1_score(Arousal_Test_dwt_ica_binary, predictions_linear_arousal, average='weighted')
print("Arousal Linear SVM:")
print(f"Accuracy: {svmlinear_arousal_2s_dwt_ica_accuracy * 100:.2f}%")
print(f"F1 Score: {svmlinear_arousal_2s_dwt_ica_f1:.2f}")
svmrbf_arousal_2s_dwt_ica = SVC(kernel='rbf',max_iter=25000,random_state=42,verbose=True)

# Train the models
svmrbf_arousal_2s_dwt_ica.fit(X_2s_dwt_ica, Arousal_Train_dwt_ica_binary)

# Predict on new data
predictions_rbf_arousal = svmrbf_arousal_2s_dwt_ica.predict(M_2s_dwt_ica)


# Evaluate the models
svmrbf_arousal_2s_dwt_ica_accuracy = accuracy_score(Arousal_Test_dwt_ica_binary, predictions_rbf_arousal)
svmrbf_arousal_2s_dwt_ica_f1 = f1_score(Arousal_Test_dwt_ica_binary, predictions_rbf_arousal, average='weighted')



# Print the results
print("Arousal RBF SVM:")
print(f"Accuracy: {svmrbf_arousal_2s_dwt_ica_accuracy * 100:.2f}%")
print(f"F1 Score: {svmrbf_arousal_2s_dwt_ica_f1:.2f}")

[LibSVM]........................WARN: libsvm Solver reached max_iter
optimization finished, #iter = 25000
obj = -696.280861, rho = 0.519882
nSV = 1505, nBSV = 527
Total nSV = 1505


/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=25000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Arousal Linear SVM:
Accuracy: 60.62%
F1 Score: 0.46
[LibSVM]........................WARN: libsvm Solver reached max_iter
optimization finished, #iter = 25000
obj = -59113.997808, rho = 0.285999
nSV = 44827, nBSV = 43804
Total nSV = 44827


/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=25000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Arousal RBF SVM:
Accuracy: 73.20%
F1 Score: 0.70


In [39]:
svmrbf_valence_2s_dwt_ica = SVC(kernel='rbf',max_iter=25000,random_state=42,verbose=True)
svmlinear_valence_2s_dwt_ica = SVC(kernel='linear',max_iter=25000,random_state=42,verbose=True)

svmrbf_valence_2s_dwt_ica.fit(X_2s_dwt_ica, Valence_Train_dwt_ica_binary)
svmlinear_valence_2s_dwt_ica.fit(X_2s_dwt_ica, Valence_Train_dwt_ica_binary)

predictions_rbf_valence = svmrbf_valence_2s_dwt_ica.predict(M_2s_dwt_ica)
predictions_linear_valence = svmlinear_valence_2s_dwt_ica.predict(M_2s_dwt_ica)

svmrbf_valence_2s_dwt_ica_accuracy = accuracy_score(Valence_Test_dwt_ica_binary, predictions_rbf_valence)
svmrbf_valence_2s_dwt_ica_f1 = f1_score(Valence_Test_dwt_ica_binary, predictions_rbf_valence, average='weighted')

svmlinear_valence_2s_dwt_ica_accuracy = accuracy_score(Valence_Test_dwt_ica_binary, predictions_linear_valence)
svmlinear_valence_2s_dwt_ica_f1 = f1_score(Valence_Test_dwt_ica_binary, predictions_linear_valence, average='weighted')

print("Valence RBF SVM:")
print(f"Accuracy: {svmrbf_valence_2s_dwt_ica_accuracy * 100:.2f}%")
print(f"F1 Score: {svmrbf_valence_2s_dwt_ica_f1:.2f}")

print("Valence Linear SVM:")
print(f"Accuracy: {svmlinear_valence_2s_dwt_ica_accuracy * 100:.2f}%")
print(f"F1 Score: {svmlinear_valence_2s_dwt_ica_f1:.2f}")

[LibSVM]........................WARN: libsvm Solver reached max_iter
optimization finished, #iter = 25000
obj = -42210.938039, rho = 0.616994
nSV = 30347, nBSV = 28570
Total nSV = 30347
[LibSVM]

/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=25000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


........................WARN: libsvm Solver reached max_iter
optimization finished, #iter = 25000
obj = -392.044417, rho = 1.739434
nSV = 1305, nBSV = 212
Total nSV = 1305


/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=25000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Valence RBF SVM:
Accuracy: 81.04%
F1 Score: 0.76
Valence Linear SVM:
Accuracy: 76.84%
F1 Score: 0.67


In [40]:
svmrbf_domain_2s_dwt_ica = SVC(kernel='rbf',max_iter=25000,random_state=42,verbose=True)
svmlinear_domain_2s_dwt_ica = SVC(kernel='linear',max_iter=25000,random_state=42,verbose=True)

svmrbf_domain_2s_dwt_ica.fit(X_2s_dwt_ica, Domain_Train_dwt_ica_binary)

svmlinear_domain_2s_dwt_ica.fit(X_2s_dwt_ica, Domain_Train_dwt_ica_binary)

predictions_rbf_domain = svmrbf_domain_2s_dwt_ica.predict(M_2s_dwt_ica)
predictions_linear_domain = svmlinear_domain_2s_dwt_ica.predict(M_2s_dwt_ica)

svmrbf_domain_2s_dwt_ica_accuracy = accuracy_score(Domain_Test_dwt_ica_binary, predictions_rbf_domain)
svmrbf_domain_2s_dwt_ica_f1 = f1_score(Domain_Test_dwt_ica_binary, predictions_rbf_domain, average='weighted')

svmlinear_domain_2s_dwt_ica_accuracy = accuracy_score(Domain_Test_dwt_ica_binary, predictions_linear_domain)
svmlinear_domain_2s_dwt_ica_f1 = f1_score(Domain_Test_dwt_ica_binary, predictions_linear_domain, average='weighted')

print("Domain RBF SVM:")
print(f"Accuracy: {svmrbf_domain_2s_dwt_ica_accuracy * 100:.2f}%")
print(f"F1 Score: {svmrbf_domain_2s_dwt_ica_f1:.2f}")

print("Domain Linear SVM:")
print(f"Accuracy: {svmlinear_domain_2s_dwt_ica_accuracy * 100:.2f}%")
print(f"F1 Score: {svmlinear_domain_2s_dwt_ica_f1:.2f}")

[LibSVM]........................WARN: libsvm Solver reached max_iter
optimization finished, #iter = 25000
obj = -36471.917882, rho = 0.702257
nSV = 26088, nBSV = 24318
Total nSV = 26088
[LibSVM]

/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=25000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


........................WARN: libsvm Solver reached max_iter
optimization finished, #iter = 25000
obj = -781.388500, rho = 0.766228
nSV = 2028, nBSV = 487
Total nSV = 2028


/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=25000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Domain RBF SVM:
Accuracy: 84.60%
F1 Score: 0.80
Domain Linear SVM:
Accuracy: 80.31%
F1 Score: 0.72


In [41]:
n = int(math.sqrt(len(X_2s_dwt_ica)))
knn = neighbors.KNeighborsClassifier(n_neighbors=n, weights='uniform', p=norm)
knn_arousal_dwt_ica = knn.fit(X_2s_dwt_ica, Arousal_Train_dwt_ica_binary)
print('k_NN accuracy for test set: %f' % knn_arousal_dwt_ica.score(M_2s_dwt_ica, Arousal_Test_dwt_ica_binary))
y_true, y_pred = Arousal_Test_dwt_ica_binary, knn_arousal_dwt_ica.predict(M_2s_dwt_ica)
print(classification_report(y_true, y_pred))

knn_valence_dwt_ica = knn.fit(X_2s_dwt_ica, Valence_Train_dwt_ica_binary)
print('k_NN accuracy for test set: %f' % knn_valence_dwt_ica.score(M_2s_dwt_ica, Valence_Test_dwt_ica_binary))
y_true, y_pred = Valence_Test_dwt_ica_binary, knn_valence_dwt_ica.predict(M_2s_dwt_ica)
print(classification_report(y_true, y_pred))

knn_domain_dwt_ica = knn.fit(X_2s_dwt_ica, Domain_Train_dwt_ica_binary)
print('k_NN accuracy for test set: %f' % knn_domain_dwt_ica.score(M_2s_dwt_ica, Domain_Test_dwt_ica_binary))
y_true, y_pred = Domain_Test_dwt_ica_binary, knn_domain_dwt_ica.predict(M_2s_dwt_ica)
print(classification_report(y_true, y_pred))

k_NN accuracy for test set: 0.861988
              precision    recall  f1-score   support

           0       0.90      0.73      0.81      6678
           1       0.84      0.95      0.89     10306

    accuracy                           0.86     16984
   macro avg       0.87      0.84      0.85     16984
weighted avg       0.87      0.86      0.86     16984

k_NN accuracy for test set: 0.854746
              precision    recall  f1-score   support

           0       0.96      0.38      0.55      3907
           1       0.84      1.00      0.91     13077

    accuracy                           0.85     16984
   macro avg       0.90      0.69      0.73     16984
weighted avg       0.87      0.85      0.83     16984

k_NN accuracy for test set: 0.873822
              precision    recall  f1-score   support

           0       0.98      0.36      0.53      3315
           1       0.87      1.00      0.93     13669

    accuracy                           0.87     16984
   macro avg     

In [42]:
Val_R_dwt_ica = RandomForestRegressor(n_estimators=512, n_jobs=6,random_state=42)
Val_R_dwt_ica.fit(X_2s_dwt_ica, Valence_Train_dwt_ica_binary)
Valence_pred_dwt_ica = Val_R_dwt.predict(M_2s_dwt_ica)
Valence_pred_dwt_ica = np.where(Valence_pred_dwt_ica >= 0.5, 1, 0)
Ran_valence_dwt_ica_accuracy = accuracy_score(Valence_Test_dwt_ica_binary, Valence_pred_dwt_ica)
Ran_valence_dwt_ica_f1 = f1_score(Valence_Test_dwt_ica_binary, Valence_pred_dwt_ica, average='weighted')
print(Ran_valence_dwt_ica_accuracy,Ran_valence_dwt_ica_f1)

Aro_R_dwt_ica = RandomForestRegressor(n_estimators=512, n_jobs=6,random_state=42)
Aro_R_dwt_ica.fit(X_2s_dwt_ica, Arousal_Train_dwt_ica_binary)
Arousal_pred_dwt_ica = Aro_R_dwt_ica.predict(M_2s_dwt_ica)

Arousal_pred_dwt_ica= np.where(Arousal_pred_dwt_ica >= 0.5, 1, 0)
Ran_Arousal_dwt_ica_accuracy = accuracy_score(Arousal_Test_dwt_ica_binary, Arousal_pred_dwt_ica)
Ran_Arousal_dwt_ica_f1 = f1_score(Arousal_Test_dwt_ica_binary, Arousal_pred_dwt_ica, average='weighted')

print(Ran_Arousal_dwt_ica_accuracy,Ran_Arousal_dwt_ica_f1)

Dom_R_dwt_ica = RandomForestRegressor(n_estimators=512, n_jobs=6,random_state=42)
Dom_R_dwt_ica.fit(X_2s_dwt_ica, Domain_Train_dwt_ica_binary)
Domain_pred_dwt_ica = Dom_R_dwt_ica.predict(M_2s_dwt_ica)

Domain_pred_dwt_ica = np.where(Domain_pred_dwt_ica >= 0.5, 1, 0)
Ran_Domain_dwt_ica_accuracy = accuracy_score(Domain_Test_dwt_ica_binary, Domain_pred_dwt_ica)
Ran_Domain_dwt_ica_f1 = f1_score(Domain_Test_dwt_ica_binary, Domain_pred_dwt_ica, average='weighted')

print(Ran_Domain_dwt_ica_accuracy,Ran_Domain_dwt_ica_f1)

0.5095383890720678 0.5481291275501832
0.9570772491756948 0.9567794949940039
0.94812764955252 0.9450998569149136
